## Loading dataset

In [1]:
import pandas as pd

df = pd.read_csv("https://huggingface.co/datasets/Andyrasika/bank_transactions/raw/main/labelled_transactions.csv")

df.head()

,Date,Supplier,Description,Transaction value (£),Classification
0,15/08/2016,Creative Video Productions Ltd,Kelvin Hall,26866,Other
1,29/05/2017,John Graham Construction Ltd,Causewayside Refurbishment,74806,Building Improvement
2,29/05/2017,Morris & Spottiswood Ltd,George IV Bridge Work,56448,Building Improvement
3,31/05/2017,John Graham Construction Ltd,Causewayside Refurbishment,164691,Building Improvement
4,24/07/2017,John Graham Construction Ltd,Causewayside Refurbishment,27926,Building Improvement


## Renaming columns

In [2]:
df.rename(columns={'Date': 'date',
                   'Supplier': 'supplier',
                   'Description' : 'description',
                   'Transaction value (£)' : 'value',
                   'Classification' : 'classification',
                   }, inplace=True)

## Table creation for storing the dataset

In [3]:
DATABASE = "postgres"
USER = "postgres"
PASSWORD = "pgpassword"
HOST = "localhost"
PORT = "5432"

In [4]:
import psycopg2
conn = psycopg2.connect(database=DATABASE, user=USER, password=PASSWORD, host=HOST, port=PORT)
cursor = conn.cursor()

cursor.execute("CREATE TABLE IF NOT EXISTS transactions(id SERIAL PRIMARY KEY,date DATE,description varchar(512),value FLOAT,classification varchar(512))")
conn.commit()

## Saving dataframe into the DB

In [5]:
from sqlalchemy import create_engine

DATABASE_URL = (
    f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"
)

engine = create_engine(DATABASE_URL)
df.to_sql('transactions', engine, if_exists='replace', index = False)

101

## Checking saved data

In [6]:
cursor.execute("""
SELECT * FROM transactions LIMIT 2
""")

for row in cursor.fetchall():
    print(row)

('15/08/2016', 'Creative Video Productions Ltd', 'Kelvin Hall', 26866, 'Other')
('29/05/2017', 'John Graham Construction Ltd', 'Causewayside Refurbishment', 74806, 'Building Improvement')
